# 1. Modeling

Bron: https://iq.opengenus.org/naive-bayes-on-tf-idf-vectorized-matrix/

In [9]:
from datetime import datetime
from functools import reduce
from pprint import pprint
import json

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer

from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn import metrics

from tqdm import tqdm

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
MODEL_CHECKPOINTS_FOLDER = "../checkpoints/"

SAVE_MODEL_CHECKPOINTS = True

RANDOM_SEED = 42

In [3]:
np.random.seed(RANDOM_SEED)

## 1.1 Load data

In [4]:
df_text = pd.read_csv("../data/aapl_us_equities_news_proc_text.csv")

## 1.2 Split data

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    df_text["text"],
    df_text["target"],
    stratify=df_text["target"],
    random_state=RANDOM_SEED,
)

## 1.3 Build pipeline

Bron: https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html

Bron: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

Bron: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

Bron: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

Bron: https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [6]:
VECTORIZERS = [
    ("count_vec", CountVectorizer()),
    ("tfidf_vec", TfidfVectorizer()),
]

CLASSIFIERS = [
    ("ada_clf", AdaBoostClassifier(random_state=RANDOM_SEED)),
    ("knn_clf", KNeighborsClassifier()),
    ("lr_clf", LogisticRegression(random_state=RANDOM_SEED)),
    ("nb_clf", MultinomialNB()),
    ("rf_clf", RandomForestClassifier(random_state=RANDOM_SEED)),
    ("sgd_clf", SGDClassifier(random_state=RANDOM_SEED)),
    ("svc_clf", SVC(random_state=RANDOM_SEED)),
]

PARAMETERS = {
    # Count
    "count_vec__min_df": [1, 0.05, 0.10, 0.15, 0.20], # Default: 1
    "count_vec__max_df": [1.0, 0.85, 0.65, 0.50],     # Default: 1.0

    # TFIDF
    "tfidf_vec__min_df": [1, 0.05, 0.10, 0.15, 0.20], # Default: 1
    "tfidf_vec__max_df": [1.0, 0.85, 0.65, 0.50],     # Default: 1.0
    "tfidf_vec__norm": ["l2", "l1", None],            # Default: l2
    "tfidf_vec__use_idf": [True, False],              # Default: True
    "tfidf_vec__smooth_idf": [True, False],           # Default: True
    "tfidf_vec__sublinear_tf": [False, True],         # Default: False
}

In [7]:
%%time
timestamp = datetime.now().strftime("%-Y%m%d%H%M%S")
results = {}
models = {}

for vec_id, vec in VECTORIZERS:
    clf_pb = tqdm(CLASSIFIERS)

    for clf_id, clf in clf_pb:
        # Create pipeline
        pipeline = Pipeline([(vec_id, vec), (clf_id, clf)])

        # Select parameters
        parameters = {k: v for k, v in PARAMETERS.items() if vec_id in k or clf_id in k}
        n_params = reduce(lambda x, y: x * len(y), parameters.values(), 1)
        n_splits = 5

        # Setup search
        grid_search = GridSearchCV(
            pipeline,
            parameters,
            scoring="f1",
            return_train_score=True,
            cv=n_splits,
            n_jobs=-1,
            verbose=0,
        )

        # Log info
        clf_pb.set_description(
            f"({vec_id}, {clf_id}): np={n_params}, ns={n_splits}, tf={n_params * n_splits}",
        )
        clf_pb.refresh()

        # Train model
        grid_search.fit(X_train, y_train)

        # Evaluate model
        clf = grid_search.best_estimator_
        clf = clf.fit(X_train, y_train)

        y_test_pred = clf.predict(X_test)

        test_score = f1_score(y_test, y_test_pred)

        # Create results
        idx = np.argmax(grid_search.cv_results_["mean_test_score"])

        result = {
            "mean_train_score": grid_search.cv_results_["mean_train_score"][idx],
            "std_train_score": grid_search.cv_results_["std_train_score"][idx],
            "mean_val_score": grid_search.cv_results_["mean_test_score"][idx],
            "std_val_score": grid_search.cv_results_["std_test_score"][idx],
            "test_score": test_score,
            "params": grid_search.best_params_,
        }

        # Store result
        results[vec_id, clf_id] = result
        models[vec_id, clf_id] = clf

        if SAVE_MODEL_CHECKPOINTS:
            with open(MODEL_CHECKPOINTS_FOLDER + f"{timestamp}_{vec_id}_{clf_id}.json", "w") as handle:
                json.dump({f"{k1}_{k2}": v for (k1, k2), v in results.items()}, handle)

(count_vec, lr_clf): np=20, ns=5, tf=100:  29%|███████████████████████████▏                                                                   | 2/7 [01:56<04:48, 57.70s/it]/home/tomdewildt/Projects/nlp/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/tomdewildt/Projects/nlp/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://s

CPU times: user 4min 44s, sys: 2min 32s, total: 7min 16s
Wall time: 3h 10min 23s


## 1.4 Evaluate models

In [10]:
for (vec_id, clf_id), values in results.items():
    y_test_pred = models[vec_id, clf_id].predict(X_test)

    accuracy = accuracy_score(y_test, y_test_pred)
    f1 = values["test_score"]

    print("-" * 100 + "\n")
    print(f"Vectorizer: {vec_id}")
    print(f"Classifier: {clf_id}" + "\n")
    print(f"Params:")
    pprint(values["params"])
    print("")
    print(f"Accuracy Score: {round(accuracy, 4)}")
    print(f"F1 Score: {round(f1, 4)}")
    print("")
    print("Report:")
    print(classification_report(y_test, y_test_pred, target_names = ["Positive", "Negative"]))
    print("")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_test_pred))
    print("\n" + "-" * 100)

----------------------------------------------------------------------------------------------------

Vectorizer: count_vec
Classifier: ada_clf

Params:
{'count_vec__max_df': 0.5, 'count_vec__min_df': 1}

Accuracy Score: 0.5287
F1 Score: 0.6019

Report:
              precision    recall  f1-score   support

    Positive       0.51      0.36      0.42       207
    Negative       0.54      0.68      0.60       228

    accuracy                           0.53       435
   macro avg       0.52      0.52      0.51       435
weighted avg       0.52      0.53      0.52       435


Confusion Matrix:
[[ 75 132]
 [ 73 155]]

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------

Vectorizer: count_vec
Classifier: knn_clf

Params:
{'count_vec__max_df': 0.65, 'count_vec__min_df': 1}

Accuracy Score: 0.4989
F1 Score: 0.5514

Report:
              precis

/home/tomdewildt/Projects/nlp/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tomdewildt/Projects/nlp/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tomdewildt/Projects/nlp/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

----------------------------------------------------------------------------------------------------

Vectorizer: tfidf_vec
Classifier: nb_clf

Params:
{'tfidf_vec__max_df': 0.5,
 'tfidf_vec__min_df': 1,
 'tfidf_vec__norm': 'l2',
 'tfidf_vec__smooth_idf': True,
 'tfidf_vec__sublinear_tf': False,
 'tfidf_vec__use_idf': False}

Accuracy Score: 0.5218
F1 Score: 0.6848

Report:
              precision    recall  f1-score   support

    Positive       0.33      0.00      0.01       207
    Negative       0.52      0.99      0.68       228

    accuracy                           0.52       435
   macro avg       0.43      0.50      0.35       435
weighted avg       0.43      0.52      0.36       435


Confusion Matrix:
[[  1 206]
 [  2 226]]

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------

Vectorizer: tfidf_vec
Classifier: rf_clf

Params:


## 1.5 Find best model

In [11]:
best_score = -1.00
best_vec_id = None
best_clf_id = None
best_values = None
best_model = None

for (vec_id, clf_id), values in results.items():
    score = values["test_score"]
    if score > best_score:
        best_score = score
        best_vec_id = vec_id
        best_clf_id = clf_id
        best_values = values
        best_model = models[vec_id, clf_id]

print("-" * 100 + "\n")
print(f"Best Vectorizer: {best_vec_id}")
print(f"Best Classifier: {best_clf_id}" + "\n")
print(f"Best Params:")
pprint(best_values["params"])
print("")
print(f"Mean Train Score: {round(best_values['mean_train_score'], 4)}")
print(f"Std Train Score: {round(best_values['std_train_score'], 4)}")
print(f"Mean Validation Score: {round(best_values['mean_val_score'], 4)}")
print(f"Std Validation Score: {round(best_values['std_val_score'], 4)}" + "\n")
print(f"Test Score: {round(best_values['test_score'], 4)}")
print("\n" + "-" * 100)

----------------------------------------------------------------------------------------------------

Best Vectorizer: tfidf_vec
Best Classifier: lr_clf

Best Params:
{'tfidf_vec__max_df': 1.0,
 'tfidf_vec__min_df': 1,
 'tfidf_vec__norm': 'l1',
 'tfidf_vec__smooth_idf': True,
 'tfidf_vec__sublinear_tf': False,
 'tfidf_vec__use_idf': True}

Mean Train Score: 0.6871
Std Train Score: 0.0004
Mean Validation Score: 0.6871
Std Validation Score: 0.0016

Test Score: 0.6878

----------------------------------------------------------------------------------------------------
